The idea here is to use the multiplex sample as the discovery sample, and confirm it in the simplex. For that, I'll use the samples_multiplex_all.txt for discovery, which means that the samples that could be both in multiplex or simplex will be used for discovery (160 samples raw). The replication set will be families that are simplex only (samples_simplex_only.txt, 90 samples raw).

Note that the ideas here can get blurry. If a person has ADHD in an extended family, enriched for ADHD, then it would make sense that ADHD comes from inherited CNVs. Conversely, if the family is truly simplex, we'd expect the changes to come from denovo CNVs. So, how is this a discovery/replication set? 

1) Even in extended families, we might identify denovo mutations, especially as the sample size is bigger. We could then check these mutations in the simplex families.

2) Inherited mutations might have effects in variables other than DX, so we could check that in simplex as well. 

Let's start the preprocessing then. First, make sure that no weird samples need to be thrown out:

In [11]:
import glob
ped_files = ['/data/sudregp/multiplex_simplex/multiplex.ped']
wes_prefix = ['CLIA', 'CCGO', 'WPS']
trios = []
affected = []
controls = []
samples = []
famids = []
for ped_file in ped_files:
    fid = open(ped_file, 'r')
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        # if the current ID and its parents have WES data, and the sample is 
        # not in yet
        if (fa.split('_')[0] in wes_prefix and
            mo.split('_')[0] in wes_prefix and
            sid.split('_')[0] in wes_prefix and sid not in samples):
            fam = {}
            fam['child'] = sid
            if aff == '1':
                affected.append(sid)
            else:
                controls.append(sid)
            fam['father'] = fa
            fam['mother'] = mo
            fam['famid'] = famid
            trios.append(fam)
            samples += [sid, fa, mo]
            famids.append(famid)
    fid.close()
samples = set(samples)
famids = set(famids)
kids = set(affected + controls)

print 'Unique samples:', len(samples)
print 'Unique families:', len(famids)
print 'Unique children:', len(kids)

Unique samples: 158
Unique families: 20
Unique children: 83


So, 2 out of the 160 unique multiplex_all samples didn't make it. Let's see who:

In [8]:
fid = open('/home/sudregp/data/multiplex_simplex/samples_multiplex_all.txt')
m = [l.rstrip() for l in fid]
fid.close()
[s for s in m if s not in samples]

['CCGO_800940', 'CCGO_800809']

In [9]:
%%bash
grep CCGO_800940 /data/sudregp/multiplex_simplex/multiplex.ped
grep CCGO_800809 /data/sudregp/multiplex_simplex/multiplex.ped

9011	CCGO_800940	0	0	1	1
9011	CCGO_800809	CCGO_800940	0	1	1
9011	CCGO_800809	CCGO_800940	0	1	1


OK, that makes sense. They're both in a family without WES data for the mother, only dad and son. Let's add them to the exclude list, and start processing at least XHMM: **NOTE that this is the first time I'm running XHMM excluding targets based on extreme GC contents and rpeeated basis!!! Might need to re-run previous stuff as well.**

In [ ]:
# in terminal
exome_targets='/data/NCR_SBRB/simplex/SeqCapEZ_Exome_v3.0_Design_Annotation_files/SeqCap_EZ_Exome_v3_hg19_capture_targets.bed'
gatk_memory="50g"
ref_fa='/fdb/GATK_resource_bundle/hg19-2.8/ucsc.hg19.fasta'
out_dir='/data/sudregp/multiplex_simplex/xhmm'

cd $out_dir
module load GATK
module load XHMM

GATK -m ${gatk_memory} GCContentByInterval -L ${exome_targets} -R ${ref_fa} -o ./DATA.locus_GC.txt
cat ./DATA.locus_GC.txt | awk '{if ($2 < 0.1 || $2 > 0.9) print $1}' > ./extreme_gc_targets.txt

# merging all subjects in the directory
ls -1 *.sample_interval_summary > depth_list.txt;
grep -v -f ../exclude.txt depth_list.txt > depth_list2.txt

xhmm --mergeGATKdepths --GATKdepthsList=depth_list2.txt -o ./DATA.RD.txt;

# this does the same thing as the XHMM script, but it actually works in parsing 
# the base pair start and ends
cat ${exome_targets} | awk 'BEGIN{OFS="\t"; print "#CHR\tBP1\tBP2\tID"}{print $1, $2, $3, NR}' > ./EXOME.targets.reg

pseq . loc-load --locdb ./EXOME.targets.LOCDB --file ./EXOME.targets.reg --group targets --out ./EXOME.target
s.LOCDB.loc-load --noweb

# this has the same effect as the suggested command, but it actually works
pseq . loc-stats --locdb ./EXOME.targets.LOCDB --group targets --seqdb ./seqdb.hg19 --noweb | awk '{if (NR >
1) { print  $4, $10 }}' | sed 's/\.\./-/' - > ./DATA.locus_complexity.txt

cat ./DATA.locus_complexity.txt | awk '{if ($2 > 0.25) print $1}' > ./low_complexity_targets.txt

xhmm --matrix -r ./DATA.RD.txt --centerData --centerType target \
-o ./DATA.filtered_centered.RD.txt \
--outputExcludedTargets ./DATA.filtered_centered.RD.txt.filtered_targets.txt \
--outputExcludedSamples ./DATA.filtered_centered.RD.txt.filtered_samples.txt \
--excludeTargets ./extreme_gc_targets.txt --excludeTargets ./low_complexity_targets.txt \
--minTargetSize 10 --maxTargetSize 10000 \
--minMeanTargetRD 10 --maxMeanTargetRD 500 \
--minMeanSampleRD 25 --maxMeanSampleRD 200 \
--maxSdSampleRD 150

xhmm --PCA -r ./DATA.filtered_centered.RD.txt --PCAfiles ./DATA.RD_PCA

xhmm --normalize -r ./DATA.filtered_centered.RD.txt --PCAfiles ./DATA.RD_PCA \
--normalizeOutput ./DATA.PCA_normalized.txt \
--PCnormalizeMethod PVE_mean --PVE_mean_factor 0.7

xhmm --matrix -r ./DATA.PCA_normalized.txt --centerData --centerType sample --zScoreData \
-o ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
--outputExcludedTargets ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_targets.txt \
--outputExcludedSamples ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_samples.txt \
--maxSdTargetRD 30

xhmm --matrix -r ./DATA.RD.txt \
--excludeTargets ./DATA.filtered_centered.RD.txt.filtered_targets.txt \
--excludeTargets ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_targets.txt \
--excludeSamples ./DATA.filtered_centered.RD.txt.filtered_samples.txt \
--excludeSamples ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt.filtered_samples.txt \
-o ./DATA.same_filtered.RD.txt

xhmm --discover -p params.txt -r ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
    -R ./DATA.same_filtered.RD.txt -c ./DATA.xcnv -a ./DATA.aux_xcnv -s ./DATA

xhmm --genotype -p params.txt -r ./DATA.PCA_normalized.filtered.sample_zscores.RD.txt \
    -R ./DATA.same_filtered.RD.txt -g ./DATA.xcnv -F $ref_fa -v ./DATA.vcf



Checking if anyone got removed:

In [12]:
%%bash
cd ~/data/multiplex_simplex/xhmm/
ls -ltr DATA.filtered_centered.RD.txt.filtered_samples.txt

-rw-rw---- 1 sudregp sudregp 0 Jan 25 15:37 DATA.filtered_centered.RD.txt.filtered_samples.txt
